## This notebook is created for Applied Data Science Capstone Project Week 3 assignment question 3

In [1]:
# Import Modules
import pandas as pd
import numpy as np

### Copy and summary of the notebook used for Question 1.  
## Scroll down for Question 3

In [2]:
# Load the data
toronto = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

# Create the dataframe
df_toronto= pd.DataFrame(toronto[0], columns = ['Postcode' , 'Borough', 'Neighbourhood'])


# Rename Postcode column to PostalCode and reset index
df_toronto.rename(columns={"Postcode":'PostalCode'}, inplace=True)
df_toronto.rename(columns={"Neighbourhood":'Neighborhood'}, inplace=True) 
df_toronto=df_toronto.reset_index(drop=True)

# Replace unassigned Neigborhoods with the name of the Borough
df_toronto.loc[df_toronto.Neighborhood.str.contains('Not assigned'), 'Neighborhood'] = \
    df_toronto.loc[df_toronto.Neighborhood.str.contains('Not assigned'), 'Borough']

# Clean up 'Not Assigned Boroughs'
df_toronto.drop(df_toronto.loc[df_toronto['Borough']=='Not assigned'].index, inplace=True)

# Merging Neigborhoods that share the same Borough and PostalCode
df_toronto = (df_toronto.groupby(['PostalCode', 'Borough'])['Neighborhood']
      .apply(lambda x: "{}".format(', '.join(x))).reset_index())

# Display first 10 records
df_toronto.head(10)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [3]:
# Display the shape of the dataframe before starting answering Question 2
df_toronto.shape

(103, 3)

### Copy and summary of the notebook used for Question 2.  
## Scroll down for Question 3

In [4]:
# Load the GEO data
df_tor_lat_lon = pd.read_csv('Geospatial_Coordinates.csv')

# Add Lat an Long to df_toronto dataframe
df_toronto.reset_index(drop=True, inplace=True)
df_toronto['Latitude'] = -99999.9
df_toronto['Longitude'] = -99999.9

for i in range(df_toronto.shape[0]):
    postalcode = df_toronto.loc[i, 'PostalCode']
    lat = df_tor_lat_lon.loc[df_tor_lat_lon['Postal Code'] == postalcode, 'Latitude'].squeeze()
    lng = df_tor_lat_lon.loc[df_tor_lat_lon['Postal Code'] == postalcode, 'Longitude'].squeeze()
    df_toronto.loc[i, 'Latitude'] = lat
    df_toronto.loc[i, 'Longitude'] = lng
    
# Display the dataframe
df_toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


#### Coordintates for Queen's Park Downtown must be adjusted before answering Question 3, because of error in dataset

In [5]:
# PostalCode M7A is assigned to wrong Borough
df_toronto[df_toronto.PostalCode=='M7A']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
85,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [6]:
# PostalCode M9A is assigned to wrong Borough
df_toronto[df_toronto.PostalCode=='M9A']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242


In [7]:
# Assign right Borough to index 85 and 93
df_toronto.Borough[85]='Downtown Toronto'
df_toronto.Borough[93]="Queen's Park"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
# Check if correction was done
df_toronto[df_toronto.PostalCode=='M7A']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
85,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


In [17]:
# Check if correction was done
df_toronto[df_toronto.PostalCode=='M9A']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
93,M9A,Queen's Park,Queen's Park,43.667856,-79.532242


# Answering Question 3

#### Import modules

In [8]:
import json # library to handle JSON files

%pip install geopy

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Note: you may need to restart the kernel to use updated packages.
Libraries imported.


#### Latitude and Longitude for Toronto, CA

In [19]:
latitude = 43.651070
longitude = -79.347015

#### Create map of Toronto, CA using Latitude and Longitude values

In [20]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}, {}, {}'.format(neighborhood, borough, lat, lng)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Decrease the dataset to only Downtown district

In [21]:
df_downtown=df_toronto[df_toronto.Borough =='Downtown Toronto']
df_downtown=df_downtown.reset_index(drop=True)
df_downtown.shape

(19, 5)

#### Create map of Downtown Toronto, CA using Latitude and Longitude values

In [22]:
# create map of New York using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_downtown['Latitude'], df_downtown['Longitude'], df_downtown['Borough'], df_downtown['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

#### Define Foursquare Credentials and Version

In [ ]:
CLIENT_ID = 'xxxxxxxxxxxxxxxxxxxxxxxxx' # your Foursquare ID
CLIENT_SECRET = 'yyyyyyyyyyyyyyyyyyyyyyyyyy' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#### Explore a central neigborhood in the downtown dataset...

In [15]:
df_downtown[df_downtown.Neighborhood=='First Canadian Place, Underground city']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
16,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.38228


#### ...and retrieve it's Latitude and Longitude

In [24]:
neighborhood_latitude = df_downtown.loc[16, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_downtown.loc[16, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_downtown.loc[16, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of First Canadian Place, Underground city are 43.6484292, -79.3822802.


In [25]:
# Fill the lat an lon variables with a central downtown location
latitude = neighborhood_latitude 
longitude = neighborhood_longitude

#### Explore the nearby venues within 500 meters and a limit of 100

In [ ]:
# Get the url
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print(url)

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df8d3fdad1ab4002430a2a4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Financial District',
  'headerFullLocation': 'Financial District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 170,
  'suggestedBounds': {'ne': {'lat': 43.652929204500005,
    'lng': -79.37607280281344},
   'sw': {'lat': 43.6439291955, 'lng': -79.38848759718655}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05df964a52059f620e3',
       'name': 'Canoe',
       'location': {'address': '66 Wellington St West',
        'crossStreet': 'at Bay Street',
        'lat': 43.647452066183476,
        'lng': -79.38132001815676,
        'labeledL

#### Create a function to repeat this for all Toronto Downtown neighborhoods

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Run the above function on each neighborhood and create a new dataframe called downtown_venues

In [29]:
downtown_venues = getNearbyVenues(names=df_downtown['Neighborhood'],
                                   latitudes=df_downtown['Latitude'],
                                   longitudes=df_downtown['Longitude']
                                  )


Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Queen's Park


#### Check the size of the newly created dataframe

In [30]:
print(downtown_venues.shape)
downtown_venues.head()

(1317, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown, St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner


#### Check how many venues were returned for each neighborhood

In [31]:
downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


#### How many unique categories of venues did we find?

In [32]:
print('There are {} uniques categories.'.format(len(downtown_venues['Venue Category'].unique())))

There are 206 uniques categories.


#### Analyze each neighborhood

In [33]:
# one hot encoding
downtown_onehot = pd.get_dummies(downtown_venues[['Venue Category']],prefix="",prefix_sep=" ")

# add neighborhood column back to dataframe
downtown_onehot['Neighborhood'] = downtown_venues['Neighborhood'] 


# move neighborhood column to the first column
fixed_columns = [downtown_onehot.columns[-1]] + list(downtown_onehot.columns[:-1])
downtown_onehot = downtown_onehot[fixed_columns]

downtown_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Rosedale,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"Cabbagetown, St. James Town",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [34]:
downtown_grouped = downtown_onehot.groupby('Neighborhood').mean().reset_index()
downtown_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.000000,0.00,0.00,0.00,0.011905,0.000000,0.000000,0.011905,0.000000,0.011905
5,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010638,0.00,0.00,0.00,0.031915,0.000000,0.042553,0.010638,0.000000,0.000000
6,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Church and Wellesley,0.012048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012048,0.000000,...,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.012048,0.000000,0.012048,0.012048
8,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.00,0.00,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
9,"Design Exchange, Toronto Dominion Centre",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00,0.00,0.01,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000


In [35]:
downtown_grouped.shape

(19, 207)

#### Let's print each neighborhood along with the top 5 most common venues

In [36]:
num_top_venues = 5

for hood in downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = downtown_grouped[downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
               venue  freq
0        Coffee Shop  0.07
1               Café  0.05
2         Steakhouse  0.04
3    Thai Restaurant  0.04
4   Asian Restaurant  0.03


----Berczy Park----
             venue  freq
0      Coffee Shop  0.09
1   Farmers Market  0.04
2         Beer Bar  0.04
3             Café  0.04
4      Cheese Shop  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                  venue  freq
0        Airport Lounge  0.13
1       Airport Service  0.13
2      Airport Terminal  0.13
3   Rental Car Location  0.07
4              Boutique  0.07


----Cabbagetown, St. James Town----
          venue  freq
0   Coffee Shop  0.09
1    Restaurant  0.07
2          Café  0.05
3   Pizza Place  0.05
4        Bakery  0.05


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.14
1       Sandwich Place  0.05
2   Italian Restaurant  0.05
3         Burger Join

### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.


In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
downtown_venues_sorted = pd.DataFrame(columns=columns)
downtown_venues_sorted['Neighborhood'] = downtown_grouped['Neighborhood']

for ind in np.arange(downtown_grouped.shape[0]):
    downtown_venues_sorted.iloc[ind, 1:] = return_most_common_venues(downtown_grouped.iloc[ind, :], num_top_venues)

downtown_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Bakery,Restaurant,Burger Joint,Sushi Restaurant,Salad Place
1,Berczy Park,Coffee Shop,Steakhouse,Cheese Shop,Beer Bar,Farmers Market,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Airport
3,"Cabbagetown, St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Flower Shop,Bakery,General Entertainment,Jewelry Store
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Ice Cream Shop,Burger Joint,Café,Gym / Fitness Center,Bubble Tea Shop,Salad Place,Bar


### Clustering Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [39]:
# set number of clusters
kclusters = 5

downtown_grouped_clustering = downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 4, 2, 0, 0, 0], dtype=int32)

#### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [40]:
# add clustering labels
downtown_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

downtown_merged = df_downtown

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
downtown_merged = downtown_merged.join(downtown_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

downtown_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,1,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
1,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675,0,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Flower Shop,Bakery,General Entertainment,Jewelry Store
2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym
3,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Chocolate Shop
4,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Bubble Tea Shop,Diner,Bookstore,Pizza Place


### Plot clusters on a map

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(downtown_merged['Latitude'], downtown_merged['Longitude'], downtown_merged['Neighborhood'], downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###  Examine Clusters

#### Cluster 0 - Downtown Working Zone
#### Most common venues are coffee shops and cafe's

In [45]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 0, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Cabbagetown, St. James Town",0,Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Flower Shop,Bakery,General Entertainment,Jewelry Store
2,Church and Wellesley,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Gastropub,Men's Store,Mediterranean Restaurant,Hotel,Gym
3,Harbourfront,0,Coffee Shop,Park,Bakery,Pub,Café,Restaurant,Mexican Restaurant,Breakfast Spot,Theater,Chocolate Shop
4,"Ryerson, Garden District",0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Bubble Tea Shop,Diner,Bookstore,Pizza Place
5,St. James Town,0,Café,Coffee Shop,Restaurant,Clothing Store,Hotel,Cosmetics Shop,American Restaurant,Beer Bar,Italian Restaurant,Diner
6,Berczy Park,0,Coffee Shop,Steakhouse,Cheese Shop,Beer Bar,Farmers Market,Seafood Restaurant,Cocktail Bar,Bakery,Café,French Restaurant
7,Central Bay Street,0,Coffee Shop,Sandwich Place,Italian Restaurant,Ice Cream Shop,Burger Joint,Café,Gym / Fitness Center,Bubble Tea Shop,Salad Place,Bar
8,"Adelaide, King, Richmond",0,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Bakery,Restaurant,Burger Joint,Sushi Restaurant,Salad Place
9,"Harbourfront East, Toronto Islands, Union Station",0,Coffee Shop,Aquarium,Hotel,Café,Brewery,Italian Restaurant,Restaurant,Scenic Lookout,Fried Chicken Joint,Baseball Stadium
10,"Design Exchange, Toronto Dominion Centre",0,Coffee Shop,Café,Hotel,Restaurant,Bar,American Restaurant,Bakery,Deli / Bodega,Seafood Restaurant,Gastropub


#### Cluster 1 - Living Zone Rosedale

In [46]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 1, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Rosedale,1,Park,Playground,Trail,Deli / Bodega,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store


#### Cluster 2 - Airport Zone

In [47]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 2, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Christie,2,Grocery Store,Café,Park,Candy Store,Diner,Italian Restaurant,Restaurant,Baby Store,Athletics & Sports,Coffee Shop


#### Cluster 3 - Living Zone

In [48]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 3, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"CN Tower, Bathurst Quay, Island airport, Harbo...",3,Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Bar,Boutique,Boat or Ferry,Rental Car Location,Sculpture Garden,Airport


#### Cluster 4 - Living Zone Christie

In [49]:
downtown_merged.loc[downtown_merged['Cluster Labels'] == 4, downtown_merged.columns[[2] + list(range(5, downtown_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"Harbord, University of Toronto",4,Café,Bar,Bakery,Japanese Restaurant,Restaurant,Theater,Sandwich Place,Italian Restaurant,Bookstore,Comfort Food Restaurant
13,"Chinatown, Grange Park, Kensington Market",4,Café,Bar,Mexican Restaurant,Vietnamese Restaurant,Coffee Shop,Bakery,Chinese Restaurant,Dumpling Restaurant,Vegetarian / Vegan Restaurant,Donut Shop
